In [1]:
import os
import json
from tqdm import tqdm
import pandas as pd
from mmdet.apis import init_detector, inference_detector

In [ ]:
root_path = '/data/ephemeral/home/sejongmin/dataset/'
json_file_path = 'test.json'

In [7]:
config_file = '/data/ephemeral/home/sejongmin/mmdetection/work_dirs/default/default_config.py'
checkpoint_file = '/data/ephemeral/home/sejongmin/mmdetection/work_dirs/default/epoch_3.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

Loads checkpoint by local backend from path: /data/ephemeral/home/sejongmin/mmdetection/work_dirs/default/epoch_3.pth


In [13]:
with open(os.path.join(root_path, json_file_path), 'r') as f:
    data = json.load(f)

prediction_strings = []
file_names = []

for image_info in tqdm(data['images']):
    prediction_string = ''
    file_name = image_info['file_name']
    
    result = inference_detector(model, os.path.join(root_path, file_name))
    
    bboxes = result.pred_instances.bboxes.cpu().numpy()
    scores = result.pred_instances.scores.cpu().numpy()
    class_ids = result.pred_instances.labels.cpu().numpy()

    # 결과 처리
    for i in range(len(scores)):
        score = scores[i]
        x_min, y_min, x_max, y_max = bboxes[i]
        
        prediction_string += f'{class_ids[i]} {score:.2f} {x_min:.2f} {y_min:.2f} {x_max:.2f} {y_max:.2f} '
    prediction_strings.append(prediction_string)
    file_names.append(file_name)

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

output_path = './work_dirs/default'
os.makedirs(output_path, exist_ok=True) 
submission.to_csv(os.path.join(output_path, 'submission.csv'), index=False)
submission.head()

  0%|          | 0/4871 [00:00<?, ?it/s]

100%|██████████| 4871/4871 [05:16<00:00, 15.38it/s]


,PredictionString,image_id
0,7 0.31 94.81 404.34 349.46 702.73 7 0.29 441.0...,test/0000.jpg
1,1 0.19 367.02 323.83 714.00 673.42 0 0.18 361....,test/0001.jpg
2,7 0.24 255.12 283.44 898.41 775.71 1 0.21 91.9...,test/0002.jpg
3,1 0.11 639.29 806.91 1024.00 957.59 7 0.09 629...,test/0003.jpg
4,1 0.24 251.09 554.71 599.33 744.20 1 0.20 110....,test/0004.jpg
